In [22]:
import pandas as pd
import json

args = [json.loads(ln) for ln in open("../../data/train_cmv_cleaned.jsonl")]
args = pd.DataFrame(args)

hate = open("../../data/CONAN.json")
conan = [json.loads(ln) for ln in open("../../data/CONAN.json")]

hate = [i for i in conan[0]["conan"]]
#hate = pd.DataFrame(hate)

list(hate.hateSpeech)

['According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
 "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
 'A school in Birningham is still separating girls from boys. This information come from a Ofsted report. A high court said in 2017 that this type of action is illegal.',
 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
 "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
 'A school in Birningham is still separating girls from boys. This information come from a Ofsted report. A high court said in 2017 that this type of action is illegal.',
 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this 

In [24]:
corpus = sorted(set(args.titles.tolist()))
corpus

hate_corpus = sorted(set(hate.hateSpeech.tolist()))

In [25]:
### COMMON SENSE TOPIC LABELS ###
import re

#TODOs: Prune and Add CauseNet Topics
cs_controversial_topics = [ln.strip() for ln in open("../../data/controversial_topics.txt")]
cs_debate_topics = [ln.strip() for ln in open("../../data/IBM_debator_topics.txt")]
cs_debate_topics_ = [ln.strip() for ln in open("../../data/IBM_debator_2.txt")]

topics = []
topics.extend(cs_controversial_topics)
topics.extend(cs_debate_topics)
topics.extend(cs_debate_topics_)

In [32]:
# Semantic-Search, Cosine Similarity
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = topics
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = sorted(set(args.titles.tolist()))
arg_topics = dict((k, []) for k in queries)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(1, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    for score, idx in zip(top_results[0], top_results[1]):
        #print(corpus[idx], "(Score: {:.4f})".format(score))
        arg_topics[query].append(corpus[idx])

In [33]:
arg_topics

{'. I am insensitive to the Gaming Industrys portrayal of women minorities and how it deals with other socially sensitive topics.': ['Sexism in video gaming'],
 '. I believe in same sex and traditional marriage but not other types of marriage': ['Same-sex marriage'],
 '. I believe the CBS show Elementary Is worth more praise and acclaim than the BBC version Sherlock': ['Criticism of the BBC'],
 'A Macbook is a better long term investment than a Windows computer.': ['Criticism of Windows Vista'],
 'A Mexican Border Fence Would Be an Ineffective Deterrent and a Colossal Waste of Taxpayer Money and Resources': ['Mexico–United States barrier'],
 'A Wired Mouse is better than a Wireless mouse.': ['Wireless'],
 'A banana is not a good method of portraying scale.': ['Principle'],
 'A comic book priced at . or . is disproportionately expensive to the amount of entertainment that can be garnered from it.': ['Predatory pricing'],
 'A congressional reboot would breathe new life into the legislati

In [45]:
fout = open("../../data/claim_topics.jsonl", "w")

for k, v in arg_topics.items():
    fout.write(json.dumps({
        "claim": k,
        "topic": v
    }))
    fout.write("\n")

In [26]:
# # Semantic-Search, Cosine Similarity
# from sentence_transformers import SentenceTransformer, util
# import torch

# embedder = SentenceTransformer('all-MiniLM-L6-v2')

# # Corpus with example sentences
# corpus = topics
# corpus_embeddings = embedder.encode(hate_corpus, convert_to_tensor=True)

# # Query sentences:
# queries = sorted(set(hate.hateSpeech.tolist()))
# hate_topics = dict((k, []) for k in queries)

# # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
# top_k = min(1, len(corpus))
# for query in queries:
#     query_embedding = embedder.encode(query, convert_to_tensor=True)

#     cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
#     top_results = torch.topk(cos_scores, k=top_k)

#     for score, idx in zip(top_results[0], top_results[1]):
#         #print(corpus[idx], "(Score: {:.4f})".format(score))
#         hate_topics[query].append(corpus[idx])

In [27]:
# hate_topics

{'"islamophobia" is a way to take away our freedom of expression.': ['2003 invasion of Iraq'],
 '"l\'islamophobie" c\'est les musulmans qui veulent empêcher le blasphème.': ['Abortion'],
 '"l\'islamophobie" c\'est un moyen de nous retirer notre liberté d\'expression.': ['Affirmative action'],
 '#saville town looks like a city of Afghanistan or Pakistan more than an English City. What do you think?': ['African National Congress'],
 '#visitmymosque. But do not ask any questions about Muslim grooming gangs targeting white schoolgirls in Rotherham, Rochdale, Tyneside, Teeside etc.': ['Alberta separatism'],
 '#visitmymosque.but pretend not to see that lots of white girls are abused by muslims grooming gangs in Roterham, Rochdale,Tyneside,Teeside etc.': ['American Civil Liberties Union'],
 "(en réagissant à une attaque terroriste:) bien sûr encore un blond aux yeux bleus qui s'appelle édouard.": ['American Hunters and Shooters Association'],
 "(en réagissant à une attaque terroriste:) encore